# Advent of Code Day 7: File Size Calculation

### Part 1:

Find sum of all dirs with a size of at most 100000. Sub dirs are counted twice if they are in a dir that itself totals 100000 max.


## Solution Functions


In [78]:
from pathlib import Path

def read_traversal_commands_from(file_name: str) -> str:
    with open(file_name) as input_file:
        return input_file.read().splitlines()

def get_directory_tree_from(lines) -> dict:
    current_level = Path("") # This is what I was missing, keeping track of tree depth
    directory_tree = {}
    for line in lines:
        if '$ cd' in line:
            if '..' in line:
                current_level = current_level.parent
            else:
                directory_name = line[5:] # Always will start "$ cd "
                current_level = current_level / directory_name
                if current_level not in directory_tree:
                    directory_tree[current_level] = []
        elif 'dir' in line:
            _, directory_name = line.split(" ")
            directory_tree[current_level].append(current_level / directory_name)
        elif line[0].isnumeric():
            file_size, file_name = line.split(" ")
            directory_tree[current_level].append((file_name, int(file_size)))
    return directory_tree

def get_file_size_for(path: Path, directory_tree: dict) -> int:
    total_size = 0
    for object in directory_tree[path]:
        if isinstance(object, tuple):
            total_size += object[1]
        elif isinstance(object, Path):
            total_size += get_file_size_for(object, directory_tree)
    return total_size

def get_dir_sizes(directory_tree: dict) -> dict:
    return {directory: get_file_size_for(directory, directory_tree) for directory in directory_tree}

def get_filtered_size_for(dir_sizes: dict, threshold: int) -> int:
    return sum(dir_size for dir_size in dir_sizes.values() if dir_size < threshold)

def get_directory_to_free_up_enough_space(dir_sizes: dict, max_disk_space: int, required_disk_space: int) -> None:
    total_space_used = dir_sizes[Path("/")]
    initial_free_space = max_disk_space - total_space_used
    clearable_space = required_disk_space - initial_free_space

    min_removeable_dir = (max_disk_space, Path("Random/Path"))

    for dir in dir_sizes:
        if dir_sizes[dir] >= clearable_space:
            if dir_sizes[dir] < min_removeable_dir[0]:
                min_removeable_dir = (dir_sizes[dir], dir)
    return min_removeable_dir[0]


## Part 1 Test Cases


## Part 1 Answer


In [70]:
lines = read_traversal_commands_from("day_7_input.txt")
directory_tree = get_directory_tree_from(lines)

sizes = get_dir_sizes(directory_tree)
answer = get_filtered_size_for(sizes, 100000)

print(answer)

2031851


## Part 2 Test Cases


## Part 2 Answer


In [79]:
lines = read_traversal_commands_from("day_7_input.txt")
directory_tree = get_directory_tree_from(lines)

sizes = get_dir_sizes(directory_tree)
answer = get_directory_to_free_up_enough_space(sizes, 70000000, 30000000)

print(answer)

2568781
